# Homework 4: Can you predict the Midterm Elections?

Due: Monday, November 3, 2014 11:59 PM

<a href=https://raw.githubusercontent.com/cs109/2014/master/homework/HW4.ipynb download=HW4.ipynb> Download this assignment</a>

#### Submission Instructions
To submit your homework, create a folder named lastname_firstinitial_hw# and place your IPython notebooks, data files, and any other files in this folder. Your IPython Notebooks should be completely executed with the results visible in the notebook. We should not have to run any code. Compress the folder (please use .zip compression) and submit to the CS109 dropbox in the appropriate folder. If we cannot access your work because these directions are not followed correctly, we will not grade your work. For the competition (problem 4), we will post a link on Piazza to a Google Form for you to submit your predictions. 


---


In [1]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn
import sklearn.datasets
import sklearn.cross_validation
import sklearn.decomposition
import sklearn.grid_search
import sklearn.neighbors
import sklearn.metrics

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactor

## Introduction

**Add Introduction**

You will use the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api) to extract the polls for the current 2014 Senate Midterm Elections and provide a final prediction of the result of each state.

#### Data

We will use the polls from the [2014 Senate Midterm Elections](http://elections.huffingtonpost.com/pollster) from the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api). 

---

## Problem 1: Data Wrangling

We will read in the polls from the [2014 Senate Midterm Elections](http://elections.huffingtonpost.com/pollster) from the [HuffPost Pollster API](http://elections.huffingtonpost.com/pollster/api) and create a dictionary of DataFrames as well a master table information for each race.

#### Problem 1(a)

Read in [this JSON object](http://elections.huffingtonpost.com/pollster/api/charts/?topic=2014-senate) containing the polls for the 2014 Senate Elections using the HuffPost API. Call this JSON object `info`.  This JSON object is imported as a list in Python where each element contains the information for one race.  Use the function `type` to confirm the that `info` is a list. 

In [2]:
### Your code here ###
import json
url = requests.get('http://elections.huffingtonpost.com/pollster/api/charts/?topic=2014-senate').content
info = json.loads(url)
type(info)

list

In [3]:
info

[{u'election_date': u'2014-11-04',
  u'estimates': [{u'choice': u'Coons',
    u'first_name': u'Chris',
    u'incumbent': True,
    u'last_name': u'Coons',
    u'lead_confidence': 99.9,
    u'party': u'Dem',
    u'value': 56.0},
   {u'choice': u'Wade',
    u'first_name': u'Kevin',
    u'incumbent': False,
    u'last_name': u'Wade',
    u'lead_confidence': 0.0,
    u'party': u'Rep',
    u'value': 34.0}],
  u'id': 571,
  u'last_updated': u'2016-06-06T14:37:54.000Z',
  u'poll_count': 5,
  u'short_title': u'2014 Delaware Senate: Wade vs. Coons',
  u'slug': u'2014-delaware-senate-wade-vs-coons',
  u'state': u'DE',
  u'title': u'2014 Delaware Senate: Wade vs. Coons',
  u'topic': u'2014-senate',
  u'url': u'http://elections.huffingtonpost.com/pollster/2014-delaware-senate-wade-vs-coons'},
 {u'election_date': u'2014-11-04',
  u'estimates': [{u'choice': u'Markey',
    u'first_name': u'Ed',
    u'incumbent': True,
    u'last_name': u'Markey',
    u'lead_confidence': 99.9,
    u'party': u'Dem',
  

#### Problem 1(b)

For each element of the list in `info` extract the state. We should have one poll per state, but we do not. Why?

**Hint**: Use the internet to find out information on the races in each state that has more than one entry. Eliminate entries of the list that represent races that are not happening.

In [4]:
### Your code here ###
import collections
state = [d['state'] for d in info]
df = pd.DataFrame(state)
df.columns = ['State']
count_state = collections.Counter(df.State)
twopoll_state = []
twopoll_index = []
for s in count_state.keys():
    if count_state[s] >1:
        #print('State appear more than once: ' + str(s))
        twopoll_state.append(s)
for i,item in enumerate(info):
    if item['state'] in twopoll_state:
        twopoll_index.append(i)
        print(str(item['short_title'])+'  '+str(item['state']) + '  index: ' + str(i))
#Remove only Index 29

2014 Oklahoma Senate: Lankford vs. Johnson  OK  index: 4
2014 Oklahoma Senate: Inhofe vs. Silverstein  OK  index: 12
2014 South Carolina Senate: Scott vs. Dickerson  SC  index: 13
2014 South Carolina Senate: Graham vs. Hutto  SC  index: 14
NH Senate: Bass vs. Shaheen  NH  index: 29
2014 New Hampshire Senate  NH  index: 31


In [5]:
removeset = set([29])
infoClean = [v for i, v in enumerate(info) if i not in removeset]

** Your answer here: **

In [6]:
infoClean

[{u'election_date': u'2014-11-04',
  u'estimates': [{u'choice': u'Coons',
    u'first_name': u'Chris',
    u'incumbent': True,
    u'last_name': u'Coons',
    u'lead_confidence': 99.9,
    u'party': u'Dem',
    u'value': 56.0},
   {u'choice': u'Wade',
    u'first_name': u'Kevin',
    u'incumbent': False,
    u'last_name': u'Wade',
    u'lead_confidence': 0.0,
    u'party': u'Rep',
    u'value': 34.0}],
  u'id': 571,
  u'last_updated': u'2016-06-06T14:37:54.000Z',
  u'poll_count': 5,
  u'short_title': u'2014 Delaware Senate: Wade vs. Coons',
  u'slug': u'2014-delaware-senate-wade-vs-coons',
  u'state': u'DE',
  u'title': u'2014 Delaware Senate: Wade vs. Coons',
  u'topic': u'2014-senate',
  u'url': u'http://elections.huffingtonpost.com/pollster/2014-delaware-senate-wade-vs-coons'},
 {u'election_date': u'2014-11-04',
  u'estimates': [{u'choice': u'Markey',
    u'first_name': u'Ed',
    u'incumbent': True,
    u'last_name': u'Markey',
    u'lead_confidence': 99.9,
    u'party': u'Dem',
  

#### Problem 1(c)

Create a dictionary of pandas DataFrames called `polls` keyed by the name of the election (a string). Each value in the dictionary should contain the polls for one of the races.

In [7]:
### Your code here ###
polls_url = {i['url']+'.csv' for i in infoClean}

In [8]:
def build_frame(url):
    """
    Returns a pandas DataFrame object containing
    the data returned from the given url
    """
    source = requests.get(url).text
    
    # Use StringIO because pd.DataFrame.from_csv requires .read() method
    s = StringIO.StringIO(source)
    
    return pd.DataFrame.from_csv(s, index_col=None).convert_objects(
            convert_dates="coerce", convert_numeric=True)

In [9]:
# Makes a dictionary of pandas DataFrames keyed on election string.
polls = dict((election['slug'], build_frame(election['url']+'.csv')) for election in infoClean)

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


#### Problem 1(d)

Now create a master table information containing information about each race. Create a pandas DataFrame called `candidates` with rows containing information about each race. The `candidates` DataFrame should have the following columns: 

1. `State` = the state where the race is being held
2. `R` = name of republican candidate
3. `D` = name of non-republican candidate (Democrate or Independent) 
4. `incumbent` = R, D or NA

**Hint**: You will need a considerable amount of data wrangling for this.

In [10]:
infoClean

[{u'election_date': u'2014-11-04',
  u'estimates': [{u'choice': u'Coons',
    u'first_name': u'Chris',
    u'incumbent': True,
    u'last_name': u'Coons',
    u'lead_confidence': 99.9,
    u'party': u'Dem',
    u'value': 56.0},
   {u'choice': u'Wade',
    u'first_name': u'Kevin',
    u'incumbent': False,
    u'last_name': u'Wade',
    u'lead_confidence': 0.0,
    u'party': u'Rep',
    u'value': 34.0}],
  u'id': 571,
  u'last_updated': u'2016-06-06T14:37:54.000Z',
  u'poll_count': 5,
  u'short_title': u'2014 Delaware Senate: Wade vs. Coons',
  u'slug': u'2014-delaware-senate-wade-vs-coons',
  u'state': u'DE',
  u'title': u'2014 Delaware Senate: Wade vs. Coons',
  u'topic': u'2014-senate',
  u'url': u'http://elections.huffingtonpost.com/pollster/2014-delaware-senate-wade-vs-coons'},
 {u'election_date': u'2014-11-04',
  u'estimates': [{u'choice': u'Markey',
    u'first_name': u'Ed',
    u'incumbent': True,
    u'last_name': u'Markey',
    u'lead_confidence': 99.9,
    u'party': u'Dem',
  

In [88]:
### Your code here ### 
State=[]
R=[]
D=[]
incumbent=[]
for k in polls.keys():
    for i in infoClean:
        if k==i['slug']:
            checkin = True
            State.append(i['state'])
            tmp = i['url'].split('-')+['vs']
            j = tmp.index('vs')
            if j!=len(tmp)-1:
                R.append(tmp[j-1])
                #print(tmp[-3])
                D.append(tmp[j+1])
                #print(tmp[-1])
                for choice in i['estimates']:
                    if choice['incumbent']==True:
                        incumbent.append(choice['party'])
                        checkin = False
                        break
                if checkin:
                    incumbent.append(np.nan)
            else:
                checkDem = False
                for choice in i['estimates']:
                    if choice['party']=='Rep':
                        R.append(choice['last_name'])
                        print(choice['last_name'])
                    if choice['party']=='Dem':
                        D.append(choice['last_name'])
                        checkDem = True
                    if choice['incumbent']==True:
                        incumbent.append(choice['party'])
                        checkin = False
                if not checkDem:
                    for choice in i['estimates']:
                        if choice['party']=='Ind':
                            D.append(choice['last_name'])
                if checkin:
                    incumbent.append(np.nan)
#print(np.shape(State))
print(R)
print(D)
#print(np.shape(incumbent))
#State = [w[1] +'-'+ w[2] for w in (k.split("-") for k in polls.keys())]
#State = [w[0] if w[1] == 'senate' else w[0]+"-"+w[1] for w in (s.split('-') for s in State)]
candidates = pd.DataFrame.from_dict(dict({'State':State,'R':R,'D':D,'incumbent':incumbent}))

Enzi
[u'mcconnell', u'daines', u'tillis', u'alexander', u'land', u'sullivan', u'wehby', u'brown', u'cochran', u'herr', u'zaccaria', u'cassidy', u'sasse', u'ernst', u'inhofe', u'lankford', u'oberweis', u'risch', u'graham', u'collins', u'capito', u'cavasso', u'wade', u'gillespie', u'mcfadden', u'weh', u'perdue', u'roberts', u'rounds', u'bell', u'Enzi', u'cornyn', u'cotton', u'scott', u'gardner']
[u'grimes', u'curtis', u'hagan', u'ball', u'peters', u'begich', u'merkley', u'shaheen', u'childers', u'markey', u'reed', u'landrieu', u'domina', u'braley', u'silverstein', u'johnson', u'durbin', u'mitchell', u'hutto', u'bellows', u'tennant', u'schatz', u'coons', u'warner', u'franken', u'udall', u'nunn', u'orman', u'weiland', u'booker', u'Hardy', u'alameel', u'pryor', u'dickerson', u'udall']


In [89]:
candidates

,D,R,State,incumbent
0,grimes,mcconnell,KY,Rep
1,curtis,daines,MT,NaN
2,hagan,tillis,NC,Dem
3,ball,alexander,TN,Rep
4,peters,land,MI,NaN
5,begich,sullivan,AK,Dem
6,merkley,wehby,OR,Dem
7,shaheen,brown,NH,Dem
8,childers,cochran,MS,Rep
9,markey,herr,MA,Dem


In [ ]:
%%bash
git add HW4.ipynb
git commit -m 'Finish on dataframe part'
git push

## Problem 2: Confidence Intervals

Compute a 99% confidence interval for each state. 

#### Problem 2(a)

Assume you have $M$ polls with sample sizes $n_1, \dots, n_M$. If the polls are independent, what is the average of the variances of each poll if the true proportion is $p$?

** Your answer here: **

#### Problem 2(b)

Compute the square root of these values in Problem 2(a) for the republican candidates in each race. Then, compute the standard deviations of the observed poll results for each race. 

In [ ]:
### Your code here ###

#### Problem 2(c) 

Plot observed versus theoretical (average of the theoretical SDs) with the area of the point proportional to number of polls. How do these compare?

In [ ]:
### Your code here ###

** Your answer here: **

#### Problem 2(d)

Repeat Problem 2(c) but include only the most recent polls from the last two months. Do they match better or worse or the same? Can we just trust the theoretical values?

In [ ]:
### Your code here ###

** Your answer here: **

#### Problem 2(e)

Create a scatter plot with each point representing one state. Is there one or more races that are outlier in that it they have much larger variabilities than expected ? Explore the original poll data and explain why the discrepancy?

In [ ]:
### Your code here ###

** Your answer here: **

#### Problem 2(f)

Construct confidence intervals for the difference in each race. Use either theoretical or data driven estimates of the standard error depending on your answer to this question. Use the results in Problem 2(e), to justify your choice.


In [ ]:
### Your code here ###

# Problem 3: Prediction and Posterior Probabilities

Perform a Bayesian analysis to predict the probability of Republicans winning in each state then provide a posterior distribution of the number of republicans in the senate.

#### Problem 3(a)

First, we define a Bayesian model for each race. The prior for the difference $\theta$ between republicans and democtrats will be $N(\mu,\tau^2)$. Say before seeing poll data you have no idea who is going to win, what should $\mu$ be? How about $\tau$, should it be large or small? 

** Your answer here: **

#### Problem 3(b)

What is the distribution of $d$ conditioned on $\theta$. What is the posterior distribution of $\theta | d$? 

**Hint**: Use normal approximation. 

**Your answer here:**

#### Problem 3(c)

The prior represents what we think before hand. We do not know who is expected to win, so we assume $\mu=0$. For this problem estimate $\tau$ using the observed differences across states (Hint: $\tau$ represents the standard deviation of a typical difference). Compute the posterior mean for each state and plot it against original average. Is there much change? Why or why not? 

In [ ]:
### Your code here ###

**Your answer here:**

#### Problem 3(d)

For each state, report a probabilty of Republicans winning. How does your answer here compare to the other aggregators?

In [ ]:
### Your code here ###

**Your answer here:**

#### Problem 3(e)

Use the posterior distributions in a Monte Carlo simulation to generate election results. In each simulation compute the total number of seats the Republican control. Show a histogram of these results.

In [ ]:
### Your code here ###

## Problem 4

Predict the results for the 2014 Midterm Elections. We will have a three competitions with the terms for scoring entries described above. For both questions below, **explain** or provide commentary on how you arrived at your predictions including code. 

**Hint**: Use election results from 2010, 2012 to build and test models.

#### Problem 4(a)

Predict the number of Republican senators. You may provide an interval. Smallest interval that includes the election day result wins. 

**Note**: we want the total so add the numbers of those that are not up for election.

In [ ]:
### Your code here ###

**Provide an explanation of methodology here**:

#### Problem 4(b)

Predict the R-D difference in each state. The predictions that minimize the residual sum of squares between predicted and observed differences wins.

In [ ]:
### Your code here ###

**Provide an explanation of methodology here**:

#### Problem 4(c)

Report a confidence interval for the R-D difference in each state. If the election day result falls outside your confidence interval in more than two states you are eliminated. For those surviving this cutoff, we will add up the size of all confidence intervals and sum. The smallest total length of confidence interval wins. 

**Note**: you can use Bayesian credible intervals or whatever else you want. 

In [ ]:
### Your code here ###

**Provide an explanation of methodology here**:

# Submission Instructions

To submit your homework, create a folder named **lastname_firstinitial_hw#** and place your IPython notebooks, data files, and any other files in this folder. Your IPython Notebooks should be completely executed with the results visible in the notebook. We should not have to run any code.  Compress the folder (please use .zip compression) and submit to the CS109 dropbox in the appropriate folder. *If we cannot access your work because these directions are not followed correctly, we will not grade your work.*
